<a href="https://colab.research.google.com/github/ronaldo-fs/signature-detection/blob/main/new_signature_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reconhecimento de assinaturas

### Reconhecimento de assinaturas

Reconhecer e entender as características exclusivas de uma assinatura em um documento envolve detectar padrões, formas e características específicas que a diferenciam de outros elementos, como texto ou desenhos.

Ao lidar com um problema de reconhecimento de assinaturas, algumas premissas e hipóteses podem ser consideradas, incluindo:
1.   **Estabilidade da assinatura:** Cada assinatura mantém características consistentes que permitem sua distinção de outras.
2.   **Qualidade da imagem:** A qualidade das imagens é adequada para garantir uma identificação precisa.
3.   **Representatividade do conjunto de dados:** O conjunto de dados utilizado reflete adequadamente a diversidade de assinaturas reais.
4.   **Uniformidade do meio de assinatura:** As assinaturas são examinadas em um meio uniforme, minimizando variações.
5.   **Ausência de manipulação maliciosa:** Não há tentativas deliberadas de adulterar ou falsificar assinaturas.

Ao realizar a busca de documentos com assinaturas para seleção de dados, algumas restrições ou condições para garantir a qualidade e representatividade do conjunto de dados, foram:
1.  **Variedade de fontes:** Buscar documentos de diferentes tipos, como contratos, formulários e recibos.
2.  **Amostragem representativa:** Selecionar uma amostra diversificada que capture diferentes estilos de assinatura.
3.  **Tamanho da amostra:** Escolher um tamanho suficiente para representar a diversidade, mas que seja gerenciável para análise.
4.  **Consentimento e privacidade:** Garantir consentimento e proteger a privacidade dos indivíduos cujas assinaturas estão sendo utilizadas.

##### Importação das bibliotecas

In [ ]:
# Importação das bibliotecas
import xml.etree.ElementTree as ET
import os, sys, json, shutil
import tensorflow as tf
import pandas as pd
import numpy as np
import locale
from glob import glob
from google.colab import userdata
from IPython.display import display, Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

### Sobre o conjunto de dados:

São 1290 imagens TIFF de documentos acompanhadas por 1290 arquivos XML. Cada arquivo XML contém informações específicas relacionadas a uma imagem TIFF. Além disso, o texto está em fonte preta sobre fundo branco.
*   Algumas informações sobre o XML podem incluir:
  *   Etiqueta DL_PAGE:
      *   `src`, qual arquivo de imagem corresponde às informações do XML
      *   Altura e largura da página
  *   Etiqueta DL_ZONE:
      *   `gedi_type`, tipo de objeto detectado, como "DLLogo" ou "DLSignature"
      *   Coordenadas (linha, coluna, altura e largura) do objeto

De 1290 imagens TIFF:
*   430 imagens não tiveram objetos detectados
*   373 imagens tiveram 1 assinatura detectada
*   84 imagens tiveram 1 logo de empresa detectado
*   306 imagens tiveram 1 assinatura e 1 logo de empresa detectados
*   55 imagens tiveram 2 assinaturas detectadas
*   42 imagens tiveram mais de 2 objetos detectados

##### Download do dataset

Para conectar o Kaggle ao Google Colab e acessar um dataset, as seguintes ações foram realizadas:
1.   **Geração da chave API do Kaggle:**
  *   Isso resultará no download do arquivo `kaggle.json`, que contém as credenciais de API.
  *   As credenciais foram armazenadas como variáveis de segurança no projeto.
2.   **Autenticação no Google Colab:**
  *   Configure as credenciais para uso no ambiente:

In [ ]:
# Caminho completo do diretório .kaggle
kaggleFolder = os.path.expanduser('~/.kaggle')

# Verifica se o diretório existe
if not os.path.exists(kaggleFolder):
    # Cria o diretório se não existir
    os.makedirs(kaggleFolder)

# Caminho completo do arquivo JSON de autenticação
kaggleAuthFile = os.path.join(kaggleFolder, 'kaggle.json')

# Cria o arquivo JSON de autenticação
with open(kaggleAuthFile, 'w') as jsonFile:
    json.dump({
        "username": userdata.get('kaggleUsr'),
        "key": userdata.get('kaggleKey')
    }, jsonFile)

# Define as permissões do arquivo
os.chmod(kaggleAuthFile, 0o600)

print("Arquivo JSON de autenticação criado com sucesso em", kaggleAuthFile)

Arquivo JSON de autenticação criado com sucesso em /root/.kaggle/kaggle.json


3.   **Instalação da biblioteca do Kaggle:**
  *   Instale a biblioteca do Kaggle no Colab usando o seguinte comando:

In [ ]:
# Instalar o pacote kaggle
!pip install kaggle

4.   **Download e acesso ao dataset:**
  *   Acesse o dataset desejado no Kaggle [clicando aqui](https://www.kaggle.com/datasets/kaiquanmah/tobacco800-with-ground-truth)
  *   Realize o download diretamente no diretório `./datasets/raw/` usando:

In [ ]:
# Excluir diretório existente
if os.path.exists("/content/datasets/"):
    shutil.rmtree("/content/datasets/")

# Criar diretório
os.makedirs("/content/datasets/raw/")

# Fazer download do dataset
!kaggle datasets download -d kaiquanmah/tobacco800-with-ground-truth --path /content/datasets/raw --force

 49% 16.0M/32.8M [00:00<00:00, 76.9MB/s]
100% 32.8M/32.8M [00:00<00:00, 120MB/s] 


4.  
  *   Descompacte o arquivo baixado:

In [ ]:
!unzip /content/datasets/raw/tobacco800-with-ground-truth.zip -d /content/datasets/raw/tobacco800-with-ground-truth/

Archive:  /content/datasets/raw/tobacco800-with-ground-truth.zip
  inflating: /content/datasets/raw/tobacco800-with-ground-truth/Tobacc800_Groundtruth_v2.0/Tobacc800_Groundtruth_v2.0/Overview of Tobacco800 Groundtruth_v2.0.txt  
  inflating: /content/datasets/raw/tobacco800-with-ground-truth/Tobacc800_Groundtruth_v2.0/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0/aah97e00-page02_1.xml  
  inflating: /content/datasets/raw/tobacco800-with-ground-truth/Tobacc800_Groundtruth_v2.0/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0/aah97e00-page02_2.xml  
  inflating: /content/datasets/raw/tobacco800-with-ground-truth/Tobacc800_Groundtruth_v2.0/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0/aam09c00.xml  
  inflating: /content/datasets/raw/tobacco800-with-ground-truth/Tobacc800_Groundtruth_v2.0/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0/aao54e00_1.xml  
  inflating: /content/datasets/raw/tobacco800-with-ground-truth/Tobacc800_Groundtruth_v2.0/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0/a

##### Conversão do dataset Tobacco 800 para o formato YOLO v8

In [ ]:
# Define os caminhos para acessar o dataset e os arquivos associados à fonte de dados.
sourceDatasetPath = '/content/datasets/raw/tobacco800-with-ground-truth'
sourceImageFilesPath = f'{sourceDatasetPath}/Tobacco800_SinglePage/Tobacco800_SinglePage/SinglePageTIF'
sourceAnnotationFilesPath = f'{sourceDatasetPath}/Tobacc800_Groundtruth_v2.0/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0'

1.  Manipulação e análise de dados estruturados em XML, com o objetivo de extrair informações detalhadas essenciais para o treinamento do YOLO v8. Preparação do dataset e conversão de dados categóricos em formato adequado.

In [ ]:
# Lista em ordem alfabética todos os arquivos XML dentro do diretório especificado
annotationFiles = sorted(glob(f'{sourceAnnotationFilesPath}/*.xml'))

# Inicializa uma lista para armazenar informações extraídas dos arquivos XML
extractedData = []

# Coletar todas as categorias de todos os nós para treinamento do LabelEncoder
allCategories = []
for filePath in annotationFiles:
    rootElement = ET.parse(filePath).getroot()
    for node in rootElement[0][0]:
        allCategories.append(node.attrib['gedi_type'])

# Treinar o LabelEncoder com todas as categorias coletadas
labelEncoder = LabelEncoder()
labelEncoder.fit(allCategories)

# Processar cada arquivo XML
for filePath in annotationFiles:
    rootElement = ET.parse(filePath).getroot()
    fileName = rootElement[0].attrib['src']
    pageHeight = rootElement[0][0].attrib['height']
    pageWidth = rootElement[0][0].attrib['width']

    # Extrair informações de cada nó na página
    for node in rootElement[0][0]:
        nodeType = node.attrib['gedi_type']
        nodeLabel = labelEncoder.transform([nodeType])[0]

        nodeId = node.attrib['id']
        xPosition = node.attrib['col']
        yPosition = node.attrib['row']
        nodeWidth = node.attrib['width']
        nodeHeight = node.attrib['height']

        if nodeType == 'DLSignature':
            authorId = node.attrib.get('AuthorID', 'NA')
            isOverlapped = node.attrib.get('Overlapped', 'NA')
        else:
            authorId, isOverlapped = 'NA', 'NA'

        # Construir linha com os dados essenciais
        row = [fileName, pageHeight, pageWidth, authorId, isOverlapped, nodeType, nodeLabel, nodeId, xPosition, yPosition, nodeWidth, nodeHeight]
        extractedData.append(row)

2.  Avaliação da integridade dos dados extraídos, com foco em garantir que foram carregados corretamente e estão prontos para uso nas etapas subsequentes do processo.

In [ ]:
xmlExtractedData = pd.DataFrame(extractedData, columns=[
    'FileName', 'PageHeight', 'PageWidth', 'AuthorID', 'Overlapped',
    'Category', 'Labels', 'ID', 'X', 'Y', 'Width', 'Height'
])
xmlExtractedData.head(10)

,FileName,PageHeight,PageWidth,AuthorID,Overlapped,Category,Labels,ID,X,Y,Width,Height
0,aah97e00-page02_1.tif,3296,2560,NA,NA,DLLogo,0,None,1074,18,374,219
1,aah97e00-page02_2.tif,3296,2560,"Boder, J.B.",Yes,DLSignature,1,None,1409,793,659,361
2,aam09c00.tif,2292,1728,"Koplow, M.G.",Yes,DLSignature,1,None,821,1422,757,183
3,aao54e00_2.tif,3296,2560,"Landry, J.T.",Yes,DLSignature,1,None,1514,708,627,206
4,abm69c00.tif,2292,1728,"Feldman, J.D.",Yes,DLSignature,1,None,667,1279,617,110
5,acr64d00.tif,2292,1728,"Boffa, J.R.",Yes,DLSignature,1,None,886,1801,463,181
6,adh36e00-page2_1.tif,3245,2560,NA,NA,DLLogo,0,None,1181,18,359,219
7,adh36e00-page2_2.tif,3246,2560,"Krivisky, B.M.",Yes,DLSignature,1,None,1232,2399,896,431
8,adh36e00_1.tif,3245,2560,NA,NA,DLLogo,0,None,1187,18,347,219
9,adh36e00_2.tif,3246,2560,"Krivisky, B.M.",Yes,DLSignature,1,None,1239,2403,889,434


3.  Divisão do DataFrame em dois subconjuntos distintos: um para treinamento e outro para validação. Exibição das dimensões de cada conjunto para proporcionar uma visão clara da distribuição dos dados entre treinamento e validação.

In [ ]:
trainData, validData = train_test_split(xmlExtractedData, test_size=0.1, random_state=13, shuffle=True)
print(trainData.shape, validData.shape)

(1207, 12) (135, 12)


4.  Construção dos diretórios para receber os dados de treinamento e validação, organizando o ambiente necessário para a alocação estruturada dos subconjuntos de dados.

In [ ]:
!mkdir -p /content/datasets/yolo-format/tobacco800/images/train
!mkdir -p /content/datasets/yolo-format/tobacco800/labels/train
!mkdir -p /content/datasets/yolo-format/tobacco800/images/val
!mkdir -p /content/datasets/yolo-format/tobacco800/labels/val

# Define os caminhos para acessar o dataset principal e os arquivos relacionados aos subconjuntos de treinamento e validação.
yoloDatasetDirectory = '/content/datasets/yolo-format/tobacco800'

trainImagePath = f'{yoloDatasetDirectory}/images/train'
trainLabelPath = f'{yoloDatasetDirectory}/labels/train'

validImagePath = f'{yoloDatasetDirectory}/images/val'
validLabelPath = f'{yoloDatasetDirectory}/labels/val'

5.  Preparação de dados em um formato que o YOLO pode utilizar diretamente para o treinamento e validação de modelos de detecção de objetos.

In [ ]:
def segregateData(dataFrame, sourceImagePath, sourceLabelPath, targetImagePath, targetLabelPath):
  # Processamento de "nomes de arquivos" únicos da coluna 'FileName' do 'DataFrame'
  uniqueFilenames = dataFrame['FileName'].unique()

  for filename in uniqueFilenames:
    # Criando uma máscara para todas as linhas que correspondem ao nome do arquivo
    mask = dataFrame['FileName'] == filename

    # Converter os tipos de dados para o formato correto
    dataFrame.loc[mask, 'Labels'] = dataFrame.loc[mask, 'Labels'].astype(int)
    dataFrame.loc[mask, 'X'] = dataFrame.loc[mask, 'X'].astype(float)
    dataFrame.loc[mask, 'Y'] = dataFrame.loc[mask, 'Y'].astype(float)
    dataFrame.loc[mask, 'Width'] = dataFrame.loc[mask, 'Width'].astype(float)
    dataFrame.loc[mask, 'Height'] = dataFrame.loc[mask, 'Height'].astype(float)

    # Filtrar o DataFrame para obter apenas as linhas correspondentes ao nome de arquivo específico
    subset = dataFrame.loc[mask]

    # Converter o DataFrame para o formato YOLO
    yoloData = subset[['Labels', 'X', 'Y', 'Width', 'Height']].to_numpy()

    # Salvar os dados em arquivos TXT
    txtFilename = os.path.join(targetLabelPath, f"{filename.split('.')[0]}.txt")
    np.savetxt(txtFilename, yoloData, fmt=["%d", "%f", "%f", "%f", "%f"])
    shutil.copyfile(os.path.join(sourceImagePath, filename), os.path.join(targetImagePath, filename))

# Segmentação dos dados em subconjuntos de treinamento e validação
segregateData(trainData, sourceImageFilesPath, sourceAnnotationFilesPath, trainImagePath, trainLabelPath)
segregateData(validData, sourceImageFilesPath, sourceAnnotationFilesPath, validImagePath, validLabelPath)

# Número de arquivos de imagem e rótulos nos diretórios de treinamento e validação
print("Número de imagens de treinamento:", len(os.listdir(trainImagePath)))
print("Número de rótulos de treinamento:", len(os.listdir(trainLabelPath)))
print("Número de imagens de validação:", len(os.listdir(validImagePath)))
print("Número de rótulos de validação:", len(os.listdir(validLabelPath)))

Número de imagens de treinamento: 807
Número de rótulos de treinamento: 807
Número de imagens de validação: 129
Número de rótulos de validação: 129


Para este treinamento, não serão utilizados métodos de validação cruzada. Algumas razões para essa decisão são:
1.  **Natureza dos dados:** Dados de assinatura real podem ser difíceis de obter em grande quantidade, tornando a divisão em conjuntos de treinamento e teste impraticável.
2.  **Complexidade do Modelo:** Métodos simples geralmente são suficientes para a tarefa, eliminando a necessidade de ajustes complexos.
3.  **Avaliação direta:** A qualidade da extração de assinatura pode ser facilmente avaliada visualmente, sem a necessidade de métricas de validação cruzada.
4.  **Especificidade da tarefa:** A tarefa pode ser altamente específica para cada caso de uso, dependendo mais da qualidade dos dados do que da complexidade do modelo.

### Configuração de hardware, instalação e carregamento do YOLO v8

Para este treinamento, o algoritmo escolhido foi o de Reconhecimento de Padrões por meio de Redes Neurais Convolucionais (CNNs). As CNNs são altamente eficazes para tarefas de reconhecimento de padrões em imagens, e neste caso específico, optei por utilizar o YOLO v8.

Configuração do hardware (GPU) para a execução do YOLO: Acesse o menu 'Editar', depois clique em 'Configurações do notebook' e defina o acelerador de hardware para 'GPU'. Após essa configuração, vamos validar se o hardware foi configurado corretamente e está disponível para uso.

In [ ]:
gpuInfo = !nvidia-smi
gpuInfo = '\n'.join(gpuInfo)
if gpuInfo.find('failed') >= 0:
  print('Não conectado a uma GPU.')
else:
  print(gpuInfo)

Instalação do pacote `ultralytics` e das [dependências](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) necessárias para utilizar o YOLO v8 utilizando o pip e verificação do software para garantir que tudo foi configurado corretamente.

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

 Definindo o diretório onde o YOLO irá procurar pelos conjuntos de dados no formato correto.

In [ ]:
from ultralytics import settings
settings.update({
  'datasets_dir': '/content/datasets/yolo-format'
})

Carregar modelo do YOLO usando

  1.  Constrói um novo modelo a partir de um arquivo YAML que descreve a arquitetura do modelo.
  2.  Carrega um modelo pré-treinado a partir de um arquivo de pontos de verificação (checkpoint), recomendado para treinamento adicional ou inferência.
  3.  Constrói um novo modelo a partir de um arquivo YAML e, em seguida, transfere os pesos de um modelo pré-treinado especificado pelo arquivo de pontos de verificação.

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

Transferred 355/355 items from pretrained weights


Inicia o treinamento do modelo com os parâmetros fornecidos. O parâmetro `data` especifica o caminho para o arquivo de configuração do conjunto de dados YOLO (geralmente chamado de `data.yaml`). O parâmetro `task` especifica a tarefa a ser realizada, que neste caso é detecção de objetos. Outros parâmetros, como `epochs`, `imgsz` e `patience`, definem o número de épocas de treinamento, o tamanho da imagem e a paciência para o treinamento, respectivamente.

In [ ]:
results = model.train(data='/content/datasets/yolo-format/tobacco800/data.yaml', task='detect', epochs=300, imgsz=640, patience=100)

Métricas de avaliação para o modelo treinado:

*   `model.val()`: Esta função calcula as métricas de avaliação usando o conjunto de validação. Não são necessários argumentos, pois o conjunto de dados e as configurações são lembrados pelo modelo. Essa função retorna um objeto que contém várias métricas de avaliação.
  *   `metrics.box.map`: Retorna a média da precisão média (mAP) para as áreas de interseção sobre a união (IoU) variando de 50% a 95% (map50-95).
  *   `metrics.box.map50`: Retorna a média da precisão média (mAP) para a área de interseção sobre a união (IoU) de 50% (map50).
  *   `metrics.box.map75`: Retorna a média da precisão média (mAP) para a área de interseção sobre a união (IoU) de 75% (map75).
  *   `metrics.box.maps`: Retorna uma lista que contém a média da precisão média (mAP) para a área de interseção sobre a união (IoU) de 50% a 95% para cada categoria.

In [ ]:
metrics = model.val()
metrics.box.map
metrics.box.map50
metrics.box.map75
metrics.box.maps